# url: https://www.youtube.com/watch?v=HcVSSBFNcKM

## 数理最適化とは
与えられた条件の中で「最善の結果」を得るための条件を、数理的に解く手法の総称。

Excelでは「ソルバー」機能を用いると実行できる。

Pythonでは、（方法は1つではないものの）PuLPライブラリを用いると実行できる。

### 「問題」の種類
- 線形計画問題・・・制約条件が一次方程式で表されるもの
- 整数計画問題・・・↑のうち、各変数の取る値が整数に限定されるもの（ナップサック問題、など）
- 組み合わせ最適化問題・・・条件を満たす「組み合わせ」を調べるもの（巡回セールスマン問題、など）
- 非線形計画問題・・・制約条件が一次方程式で表されないもの

etc...

## 数理最適化における問題の種類

### 線形計画問題 (Linear Programming Problem, LP):
制約条件が一次方程式で表される問題です。目的関数と制約条件が線形であるという特徴があります。具体的な問題例として、生産計画や資源配分の最適化が挙げられます。

### 整数計画問題 (Integer Programming Problem, IP):
線形計画問題のうち、各変数の取る値が整数に制約される問題です。例えば、製品の数量やプロジェクトのスケジュールにおいて、変数が整数の単位でしか取り得ない場合に適用されます。具体的な問題例としては、ナップサック問題や生産ロットサイズの最適化があります。

### 組み合わせ最適化問題 (Combinatorial Optimization Problem):
条件を満たす「組み合わせ」を調べる問題です。典型的な問題には、巡回セールスマン問題、最大流問題、最小費用流問題、クリティカルパス分析などがあります。これらの問題は、組み合わせ論やグラフ理論と密接な関係があります。

### 非線形計画問題 (Nonlinear Programming Problem, NLP):
制約条件が一次方程式で表されない、つまり非線形な問題です。目的関数や制約条件が線形でない場合に適用されます。具体的な問題例として、生態系のモデリングや最適制御問題が挙げられます。※(PulPでは解けないらしい)

In [1]:
!pip install pulp

In [2]:
from pulp import *

## 線形計画問題・整数計画問題


**【例題】**

牛肉7,200g、ご飯5,400gの在庫がある。
売上を最大にするには「牛丼」「ハンバーグ定食」をそれぞれ幾つづつ作るか？

|メニュー|値段|必要な牛肉|必要なご飯|
|---|---|---|---|
|牛丼|400円|180g|90g|
|ハンバーグ定食|500円|120g|180g|


In [3]:
#モデルの定義
m = LpProblem(sense=LpMaximize) # 最大にすることを指定, 最小化: LpMinimize
# 変数 m はPuLPの LpProblem クラスのインスタンスであり、このインスタンスが最大化または最小化したい対象を表します。

#変数の定義
x = LpVariable("x", cat="Integer") # 牛丼, パラメータ内のxは誤差行, cat = で整数指定
y = LpVariable("y", cat="Integer") # ハンバーグ

#目的関数
m += 400 * x + 500 * y # 売上を最大化する式を指定

#条件(制約)
m += 180 * x + 120 * y <= 7200 # 牛肉の条件(必要な牛肉の量)
m += 90 * x + 180 * y <= 5400 # ご飯の条件(必要なご飯の量)

In [4]:
#実行
status = m.solve() # 実行
print("Status", LpStatus[status]) # 結果の確認, 解なしや条件の設定の間違いでエラーが発生する

# 結果の表示
print("Result") #Status Optimal:計算成功, 結果を確認できる, 0:解なし, 1:解あり, -n:条件の設定の間違い等のエラー
print("x", x.value()) #牛丼は30
print("y", y.value()) #ハンバーグ定食は30
print("Sales", m.objective.value()) #上記の条件での最適化された売上

Status Optimal
Result
x 30.0
y 15.0
Sales 19500.0


## 組み合わせ最適化問題

**【例題】**

300円で遠足に持っていくお菓子を選びたい。

同じお菓子は2つ以上買わないと決めた時、もっとも満足度の高い組み合わせは何か？

|お菓子|満足度|値段|
|---|---|---|
|チョコレート|4|80円|
|ポテトチップス|5|120円|
|クッキー|6|140円|
|グミ|2|60円|
|バナナ|7|150円|

In [5]:
#モデルの定義
m = LpProblem(sense=LpMaximize) # 最大化

#変数の定義(binary(0:買わない, 1:買う))
x1 = LpVariable("x1", cat="Binary") # チョコ
x2 = LpVariable("x2", cat="Binary") # ポテトチップス
x3 = LpVariable("x3", cat="Binary") # クッキー
x4 = LpVariable("x4", cat="Binary") # グミ
x5 = LpVariable("x5", cat="Binary") # バナナ

#目的関数
m += 4 * x1 + 5 * x2 + 6 * x3 + 2 * x4 + 7 * x5 # 満足度の高い組み合わせ

#条件
m += 80 * x1 + 120 * x2 + 140 * x3 + 60 * x4 + 150 * x5 <= 300 # 値段

In [6]:
#実行
status = m.solve()
print("Status", LpStatus[status])

# 結果の表示
print("Result")
print("x1", x1.value())
print("x2", x2.value())
print("x3", x3.value())
print("x4", x4.value())
print("x5", x5.value())

print("Satisfy", m.objective.value())

Status Optimal
Result
x1 1.0
x2 0.0
x3 0.0
x4 1.0
x5 1.0
Satisfy 13.0


## 以下の+=ってなに? どんな目的で使用されているの?

目的関数

    m += 400 * x + 500 * y # 売上を最大化する式を指定

条件(制約)

    m += 180 * x + 120 * y <= 7200 # 牛肉の条件(必要な牛肉の量)
    m += 90 * x + 180 * y <= 5400 # ご飯の条件(必要なご飯の量)

+= は、変数に新しい値を加算してその変数に再代入するためのショートカットです。具体的には、次のような意味があります。

    m += 400 * x + 500 * y

これは、目的関数に 400 * x + 500 * y を加える操作です。m はPuLPの LpProblem インスタンスであり、この行の目的は 400 * x + 500 * y を目的関数に追加しています。これにより、最適化の際にこの目的関数が最大化されるようになります。

同様に、制約条件も += を使用して追加されています。例えば：

    m += 180 * x + 120 * y <= 7200

この行は、制約条件を 180 * x + 120 * y <= 7200 に追加します。この制約条件は、最適化の際に考慮され、その結果として最終的な解が求められます。

+= は、数理最適化のモデリングにおいて、目的関数や制約条件を効果的かつ簡潔に追加するためによく使用される演算子です。

---

## =だけだと上書きされるから+= にしているって認識でいいの?

はい、その通りです。+= は変数に新しい値を加算し、その変数に再代入する演算子です。これにより、既存の値に新しい項を加えていく形でモデルを構築できます。

例えば、最初に目的関数を設定するときに = を使うと、既存の目的関数が上書きされます。しかし、+= を使うと、既存の目的関数に新しい項を追加していくことができます。同様に、制約条件も += を使って追加しているので、複数の制約条件が同時に考慮されることになります。

この手法は、数理最適化のモデリングにおいて柔軟性を持たせるために使われます。新しい項を加えたい場合に都度 += を使用することで、モデルを段階的に構築できます。





